# BEMM459_Week 10 Small Group Teaching (rescheduled from Week 9 cohort teaching) - MongoDB with Python and MongoEngine

### <font color="green"> Acknowledgement: Tutorial created by refering to sources including tutorialspoint.com, realypython.com and docs.mongodb.com</font>

### <font color="green"> Please refer to MongoDB commands from Week 8 cohort teaching - available under <a href=https://github.com/NavonilNM/BEMM459_RDBMS_NoSQL/blob/main/Week%208/Week%208_MONGODB%20(BEMM459%20Cohort%20Tearching).pdf>BEMM459 GitHub repository</a></font>

# 1. Installing MongoEngine

In [ ]:
# Install a pip package in the current Jupyter kernel. We are installing the mongoengine ODM package

import sys
!{sys.executable} -m pip install mongoengine

In [ ]:
import mongoengine

#Displaying the version number
mongoengine.__version__

# 2. Connectivity and new MongoDB database

In [ ]:
#Connecting to the MongoDB server using the connect() function provided by the library mongoengine
#Also creating new collection called 'mongoengineDB'

from mongoengine import connect
con=connect("mongoengineDB", host="127.0.0.1", port=7000)

# 3. Class declaration and insert operations

In [ ]:
#MongoEngine defines a Document class. 
#This is a base class whose inherited class is used to define structure and properties of collection of documents stored in MongoDB database. 
#Each object of this subclass forms Document in Collection in database.

from mongoengine import *

class Employee(Document):
    _id = StringField(required=True)
    employeeFirstName = StringField(max_length=40)
    employeeLastName = StringField(max_length=40)
    employeeSalary = IntField()

In [ ]:
# Declare object of Employee class and call save() method to store the document in a database.
emp1=Employee(_id="Emp01", employeeFirstName="MongoDB", employeeLastName="Python", employeeSalary=50000).save()

# Alternative method
emp2=Employee()
emp2._id="Emp02"
emp2.employeeFirstName="Redis"
emp2.employeeLastName="I am REDIS"
emp2.employeeSalary=60000
emp2.save()

In [ ]:
#To create a class that supports polymorphic schema, subclass it from DynamicDocument base class. 
class Project(DynamicDocument):
    _id = IntField(required=True)
    projectName = StringField()    
    
#Create an object which follows the schema for Project
prj1=Project()
prj1._id=1
prj1.projectName="MongoDB advance project"
prj1.save()

#**Important**Add another attribute to second document (example of schemaless)
prj2=Project()
prj2._id=2
prj2.projectName="REDIS advance project"
setattr(prj2, "projectLocation", "London")
prj2.save()

In [ ]:
# To be used for Reference field - see below (student has a personal tutor who is a teacher)
class Teacher(Document):
    teacherId = StringField(required=True)
    teacherFirstName=StringField()
    teacherLastName=StringField()   
     
tchr1=Teacher()
tchr1.teacherId="3"
tchr1.teacherFirstName="I am your"
tchr1.teacherLastName="Teacher"
tchr1.save()

# 4. Advance: Class declaration with MongoEngine fields and attributes
### ListField, DictField, ReferenceField
### Attributes min_value and max_value
### Embedded documents

In [ ]:
class Student(Document):
    studentid = StringField(required=True)
    studentFirstname = StringField(max_length=50)
    studentLastName = StringField(max_length=50)
    
    #The attribute min_value and max_value specifiy minimum and maximum acceptable value
    studentAge = IntField(min_value=8, max_value=16)
    
    #ListField - wraps any standard field, thus allowing multiple objects to be used as a list object in a database. 
    #This field can be used with Reference Field to implement one to many relationships.
    studentSubjects = ListField(StringField())
    
    #DictField - An object of DictField class stores a Python dictionary object (associative array)
    #Creates embedded object
    studentSkills = DictField()
    
    #ReferenceField - A MongoDB document can store reference to another document using this type of field. 
    #Allows us to implement join as in RDBMS. 
    #A ReferenceField constructor uses name of other document class (Teacher - see [3] above) as parameter
    teacherId = ReferenceField(Teacher)
    
#StringField
stud1=Student()
stud1.studentid="1"
stud1.studentFirstname= "Lost World"
stud1.studentFirstname= "Hello World"
stud1.studentAge= 15

#ListField
stud1.studentSubjects=['Programming', 'Databases', 'Maths and Stats', 'Visual Analytics']

#DictField
stud1.studentSkills['Redis']="expert"
stud1.studentSkills['MySQL']="novice"
stud1.studentSkills['SQLite']="expert"
stud1.studentSkills['MongoDB']="intermediate"

#ReferenceField
#Note: Note that ReferenceField in Student document stores _id of corresponding Teacher document.
stud1.teacherId=tchr1

#Save document
stud1.save()

# 5. One-to-many Relationships using List [] of RefereneField

In [ ]:
#You can implement one to many relationship using list of references. 
#Assuming that a student document has to be related with one or more teacher documents..
#..the Student class must have a ListField of ReferenceField instances.

#This is a slighly more complex class compared to class Student. Here teacherId is ListField() of ReferenceField()
class StudentTeacher(Document):
    studentid = StringField(required=True)
    studentFirstName = StringField(max_length=50)
    studentLastName = StringField(max_length=50)
    
    #The attribute min_value and max_value specifiy minimum and maximum acceptable value
    studentAge = IntField(min_value=8, max_value=16)
    
    #ListField - wraps any standard field, thus allowing multiple objects to be used as a list object in a database. 
    #This field can be used with Reference Field to implement one to many relationships.
    studentSubjects = ListField(StringField())
    
    #DictField - An object of DictField class stores a Python dictionary object (associative array)
    #Creates embedded object
    studentSkills = DictField()
    
    #** List of references using ReferenceField
    #** This is a slighly more complex class compared to class Student. Here teacherId is ListField() of ReferenceField()
    teacherId = ListField(ReferenceField(Teacher))
    
#First add three more teachers..refer to class Teacher under [3] above
tchr3=Teacher(teacherId="3", teacherFirstName="Teacher X", teacherLastName="Teacher Y").save()
tchr4=Teacher(teacherId="13", teacherFirstName="Teacher XX", teacherLastName="Teacher YY").save()
tchr5=Teacher(teacherId="23", teacherFirstName="Teacher XXX", teacherLastName="Teacher YYY").save()   
 
# Now adding student and creating one-to-many relationship with teacher
stud1=StudentTeacher()
stud1.studentid="515"
stud1.studentFirstName= "Lost World"
stud1.studentLastName= "Hello World"
stud1.studentAge= 15

#ListField
stud1.studentSubjects=['Programming', 'Databases', 'Maths and Stats', 'Visual Analytics']

#DictField
stud1.studentSkills['Redis']="expert"
stud1.studentSkills['MySQL']="novice"
stud1.studentSkills['SQLite']="expert"
stud1.studentSkills['MongoDB']="intermediate"

#ReferenceField
#Note: Note that ReferenceField in Student document stores _id of corresponding Teacher document.
stud1.teacherId=[tchr3,tchr4,tchr4]

#Save document
stud1.save()

In [ ]:
# Now adding another student to StudentTeacher class and creating one-to-many relationship with teacher
stud2=StudentTeacher()
stud2.studentid="60"
stud2.studentFirstName= "Mongo"
stud2.studentLastName= "King"
stud2.studentAge= 15

#ListField
stud2.studentSubjects=["Databases", "Visual Analytics"]

#DictField
stud2.studentSkills["Databases"]="expert"
stud2.studentSkills["Visual Analytics"]="novice"

#ReferenceField
#Note: Note that ReferenceField in Student document stores _id of corresponding Teacher document.
stud2.teacherId=[tchr4,tchr5]

#Save document
stud2.save()

# 6. Queries

## 6A. Querying databases and collections

In [ ]:
#Query the databases present in the server
dbs=con.list_database_names()
for db in dbs:
    print(db)

In [ ]:
#query collections in database mongoDB
collections=con["mongoengineDB"].list_collection_names()
for collection in collections:
    print (collection)

## 6B. Returning documents

In [ ]:
#First document satisfying the query is returned
#The Document class has objects attribute that enable access to objects associated with the database.
#Note in MongoEngine use .id to refer to _id

tempObj=StudentTeacher.objects
doc=tempObj.first()
print('_ID:', doc.id, 'Student ID:', doc.studentid, '  First Name:',doc.studentFirstName, '  Last Name:', doc.studentLastName, '  Age:', doc.studentAge)

## 6C. Returning embedded documents

In [ ]:
#Printing documents in collection StudentTeacher (including embedded Document StudentSkill)
#The Document class has objects attribute that enable access to objects associated with the database.
#Note in MongoEngine use .id to refer to _id
for stud_teacher in StudentTeacher.objects:
    print ('_ID:', stud_teacher.id, 'Student ID:', stud_teacher.studentid, '  First Name:',stud_teacher.studentFirstName, '  Last Name:', stud_teacher.studentLastName, '  Age:', stud_teacher.studentAge)
    
    #ListField
    print ('Student Subjects:')
    for subject in stud_teacher.studentSubjects:
        print('  ', subject)

    #DictField - embedded document
    print ('Student Skills:')
    for key,value in stud_teacher.studentSkills.items():
        print('  ', key, " : ", value)
        
    #ReferenceField    
    print ('Students and Teachers:')
    for teacher in stud_teacher.teacherId:
        print('_ID:', teacher.id, '  Teacher ID:', teacher.teacherId, 'Teacher First Name:', teacher.teacherFirstName, 'Teacher Last Name:', teacher.teacherLastName,)

## 6D. Projection

In [ ]:
#Projection
#Use field names as keyword arguments to fields() and set to 1 to include, 0 to exclude.

for stud_teacher in StudentTeacher.objects.fields(id=0,studentFirstName=1,studentLastName=1):
    print (stud_teacher.to_json())

## 6E. Use of filters and operators

In [ ]:
#Use of filter (similar to where clause in SQL). 

#Method 1: The objects attribute is a QuerySet manager. 
#It creates and returns a QuerySet when accessed. A query can be subjected to filter with the help of field names as keyword arguments.

for stud_teacher in StudentTeacher.objects(studentid="60"):
    print ('_ID:', stud_teacher.id, 'Student ID:', stud_teacher.studentid, '  First Name:',stud_teacher.studentFirstName, '  Last Name:', stud_teacher.studentLastName, '  Age:', stud_teacher.studentAge)

#Method 2 - using the filter object of QuerySet
#query=StudentTeacher.objects
#for stud_teacher in query.filter(studentid="515"):
#    print ('_ID:', stud_teacher.id, 'Student ID:', stud_teacher.studentid, '  First Name:',stud_teacher.studentFirstName, '  Last Name:', stud_teacher.studentLastName, '  Age:', stud_teacher.studentAge)

In [ ]:
#Refer to MongoEngine documentation for query operators - https://docs.mongoengine.org/guide/querying.html
#Operators other than equality may also be used in queries — just attach the operator name to a key with a double-underscore:
#E.g., __gt (greater than), __lt (less than), __gte (greater than or equal to),__in (value is in list), etc.

for stud_teacher in StudentTeacher.objects(studentAge__lt=18):
    print ('_ID:', stud_teacher.id, 'Student ID:', stud_teacher.studentid, '  First Name:',stud_teacher.studentFirstName, '  Last Name:', stud_teacher.studentLastName, '  Age:', stud_teacher.studentAge)

In [ ]:
#Example with __in[]
#Refer to MongoEngine documentation for query operators - https://docs.mongoengine.org/guide/querying.html

for stud_teacher in StudentTeacher.objects(studentFirstName__in=["Mongo", "Redis"]):
    print ('_ID:', stud_teacher.id, 'Student ID:', stud_teacher.studentid, '  First Name:',stud_teacher.studentFirstName, '  Last Name:', stud_teacher.studentLastName, '  Age:', stud_teacher.studentAge)

In [ ]:
#Example with regular expression
#Refer to MongoEngine documentation for query operators - https://docs.mongoengine.org/guide/querying.html
# exact (exact string match), iexact (exact string match - case insensitive, contains (string field contains value), etc.

for stud_teacher in StudentTeacher.objects(studentFirstName__contains="o"):
    print ('_ID:', stud_teacher.id, 'Student ID:', stud_teacher.studentid, '  First Name:',stud_teacher.studentFirstName, '  Last Name:', stud_teacher.studentLastName, '  Age:', stud_teacher.studentAge)

## 6F. Output as JSON object

In [ ]:
#to_json() mehod of Document class is used to obtain JSON version of Document

for stud_teacher in StudentTeacher.objects:
    print (stud_teacher.to_json())

In [ ]:
#to_json() mehod of Document class is used to obtain JSON version of Document
#We are importing json library to pretty print

import json

for stud_teacher in StudentTeacher.objects:
    my_json = stud_teacher.to_json()
    parsed = json.loads(my_json)
    print(json.dumps(parsed, indent=4, sort_keys=True))

# 7. Modify/Update a Document

In [ ]:
# Update document in database

#First document satisfying the query is returned
qset=StudentTeacher.objects(studentid="60")
modStudent=qset.first()

#Print before update
print('Before Update_ID:', modStudent.id, 'Student ID:', modStudent.studentid, '  First Name:',modStudent.studentFirstName, '  Last Name:', modStudent.studentLastName, '  Age:', modStudent.studentAge)

#Updating values
modStudent.update(studentid="123")
modStudent.update(studentFirstName= "Now I am 123")
modStudent.update(studentLastName= "Now I am 123")

#Print after update - note that the objectIDs are the same
qset=StudentTeacher.objects(studentid="123")
modStudent=qset.first()
print('After Update_ID:', modStudent.id, 'Student ID:', modStudent.studentid, '  First Name:',modStudent.studentFirstName, '  Last Name:', modStudent.studentLastName, '  Age:', modStudent.studentAge)


# 8. Delete Document, Collection and Database

In [ ]:
# Delete document from database
qset=StudentTeacher.objects(studentid="123")
delStudent=qset.first()

delStudent.delete()

In [ ]:
#Drops the entire collection associated with this Document type from the database.
StudentTeacher.drop_collection()

In [ ]:
#Drop database
con.drop_database("mongoengineDB")

# 9. Close Connection to the database

In [ ]:
#Closing the database connection
con.close()